# Categories

# 1. Data Preparation

1.1 import libraries

In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
import re
import nltk.stem as Stemmer
from hazm import *
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
import sklearn
from numpy import array

In [19]:
df=pd.read_csv('train_set.csv')
df

,app_id,description_fa,label
0,0,<p>بازی مین روب یک برنامه فکری است که باید مین...,1
1,1,<p>دراین بازی تعدادی عکس برای شما نشان داده می...,1
2,2,<br>تلاش نافرجام برای درک «بوفالو»<br>مرگ پایا...,7
3,3,<p>فیلم نما ، برنامه ای برای دانلود و پخش آنلا...,7
4,4,<p>* این برنامه حاوی بیش از 500 عکس و ژست برای...,7
...,...,...,...
37894,37894,<h3>بدون هیچ هزینه ای، هر هفته مسابقه</h3>\n<p...,1
37895,37895,<p>حیوانات مزرعه به کمک شما احتیاج دارند. بلوک...,1
37896,37896,<p><b>_ دوست داری عکس پروفایلت خاص باشه؟</b></...,7
37897,37897,<p> <p> <b>@@ اولین نسخه این بازی @@ </b> <br>...,1


In [20]:
df_test=pd.read_csv('test_set.csv')
df_test

,app_id,description_fa
0,40000,<h4>در این برنامه یکی از بازی های جالب میکرو ر...
1,40001,<p>بازی آموزشی میلونر زبان انگلیسی برای تقویت ...
2,40002,<p> <b>موضوع رو می بینی و کلمات رو پیدا می کنی...
3,40003,<p>نرم افزار فول تلگرام در کنار تلگرام امكانات...
4,40004,<p> فکر می کنی خیلی سریع هستی نه؟ <br> ...
...,...,...
9995,49995,<p>فالوورگیر اینستاگرام با امکانات <b>لایک </b...
9996,49996,<p><b>میخوای نامحدود آنفلو کنی ؟</b></p>\n<p><...
9997,49997,<p>یک بازی زیبا از کنسول میکرو</p>\n<p>به یاد ...
9998,49998,<p>دوست داری اطلاعات عمومی بالایی داشته باشی؟&...


In [21]:
persian_stopwords = open("StopWords.txt", encoding="utf8")
persian_stopwords = persian_stopwords.read().split("\n")

# 2. Data Preprocessing

2.1 data preprocessing functions

In [22]:
def text_cleaning(text):
    persian_stemmer = Stemmer()
    results = []
    
    text = re.sub(r'\@\w*', '', str(text))
    text = re.sub(r'$', '', str(text))
    text = re.sub(r'<(.*?)>', '', str(text))
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', str(text))
    text = re.sub(r'#', '', str(text))
    
    tokens = word_tokenize(text)
    
    for word in tokens:
        if word not in persian_stopwords and word not in string.punctuation:
            result = persian_stemmer.stem(word)
            results.append(result)
            
    return ' '.join(results)

In [23]:
def params(data):
    text = []
    for txt in data:
        text.append(" ".join(map(str, txt)))
    
    X = [text[i].split(" ") for i in range(len(text))]
    X = pad_sequences(X, padding='pre', maxlen=10000)
    
    return X

# 3. Logistic Regression

3.1 Preprocessing

In [24]:
X_train = [text_cleaning(i) for i in np.array(df['description_fa'])]
Y_train = df['label']
X_test = [text_cleaning(i) for i in np.array(df_test['description_fa'])]

tf_idf = TfidfVectorizer()

In [8]:
X_train = tf_idf.fit_transform(X_train)
X_test = tf_idf.transform(X_test)

In [9]:
# X_train, X_dev, Y_train, Y_dev = train_test_split(text, df['label'], test_size=0.1, random_state=42)

3.2 model

In [15]:
clf = LogisticRegression(random_state=0, C = 2, max_iter=1000).fit(X_train, Y_train)

In [16]:
predictions = clf.predict(X_test)
predictions

array([6, 8, 5, ..., 6, 5, 3], dtype=int64)

3.3 Result

In [ ]:
result = pd.concat([df_test['app_id'], pd.DataFrame(predictions, columns = ['label'])], axis = 1)
result.to_csv(r'Logistic_prediction.csv', index=0)
result

# SVM

In [10]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [11]:
clf = SVC(kernel = 'rbf', C=5, gamma=1)

In [12]:
clf.fit(X_train, Y_train)

SVC(C=5, gamma=1)

In [13]:
predictions = clf.predict(X_test)
predictions

array([6, 1, 5, ..., 6, 5, 5], dtype=int64)

In [14]:
result = pd.concat([df_test['app_id'], pd.DataFrame(predictions, columns = ['label'])], axis = 1)
result.to_csv(r'SVM_prediction.csv', index=0)
result

,app_id,label
0,40000,6
1,40001,1
2,40002,5
3,40003,9
4,40004,3
...,...,...
9995,49995,9
9996,49996,9
9997,49997,6
9998,49998,5
